In [50]:
# import
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from itertools import repeat

import numpy as np
import pandas as pd
from Utils import custom_data_loader, preprocess_data
from Utils.SummaryWriter import LogSummary
from Models.simpleFFBNN import SimpleFFBNN



In [2]:
def get_device():
    """Function to get the device to be used for training the model
    """
    cuda = torch.cuda.is_available()
    print("CUDA Available: ", cuda)

    if cuda:
        gpu = GPUtil.getFirstAvailable()
        print("GPU Available: ", gpu)
        torch.cuda.set_device(gpu)
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    print("Device: ", device)
    return device

device = get_device()


CUDA Available:  False
Device:  cpu


In [3]:
# load data
dataloader_train, dataloader_test, dataloader_val = preprocess_data(pd.read_csv('/Users/kristian/Documents/Skole/9. Semester/Thesis Preparation/Code/BNNs/Data/quality_of_food.csv'), batch_size = 128)


/Users/kristian/Documents/Skole/9. Semester/Thesis Preparation/Code/BNNs/Utils/config.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.X['savings'] = np.where(self.X['savings'] == 'low', 0, np.where(self.X['savings'] == 'medium', 1, 2))


In [4]:
model = SimpleFFBNN(4, 1)

In [72]:
checkpoint = torch.load('/Users/kristian/Documents/Skole/9. Semester/Thesis Preparation/Code/BNNs/trainedModels/simple_model.pth', map_location=torch.device('cpu'))
#print(checkpoint)
#model.load_state_dict(checkpoint['model'])
model.load_state_dict(checkpoint)
    

<All keys matched successfully>

In [33]:
class Saveoutput():
    def __init__(self, instances, batch_size, rounds):
        self.T = instances
        self.batch_size = batch_size
        self.outputs = []
        self.rounds = rounds
        self.counter = 0


    def __call__(self, module, module_in, module_out):
        if self.counter < 3:
            sample_data = np.random.randint(self.batch_size)
            outs = module_out.view(self.batch_size, -1)
            #outs = module_out.view(self.T, self.batch_size, -1)[:, 0, :]
            layer_size = outs.shape[1]

            
            write_summary.per_round_layer_output(layer_size, outs, self.rounds)

            self.counter += 1


    def clear(self):
        self.outputs = []
        


        

In [7]:
write_summary = LogSummary(name = 'Simple Model')

In [121]:
class runActiveLearning():
    def __init__(self, model, dataloader_train, dataloader_test, dataloader_val, epochs, rounds, learning_rate, batch_size, instances, highest_unc, seed_sample):
        self.model = model
        self.dataloader_train = dataloader_train
        self.dataloader_test = dataloader_test
        self.dataloader_val = dataloader_val
        self.epochs = epochs
        self.rounds = rounds
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.instances = instances
        self.highest_unc = highest_unc
        self.seed_sample = seed_sample


        # a set of lists to store the selected indices with highest uncertainty
        self.highest_unc = set([])
        # unexplored data
        self.unexplored_data = set(range(len(dataloader_train.dataset)))

        training_params = sum(p.numel() for p in self.model.parameters())
        print('Running Model:{}'.format(model, training_params, self.epochs, self.batch_size))

    def loadSeedModel(self):
        # load the trained model
        model = SimpleFFBNN(4, 1)
        checkpoint = torch.load('/Users/kristian/Documents/Skole/9. Semester/Thesis Preparation/Code/BNNs/trainedModels/simple_model.pth', map_location=torch.device('cpu'))
        model.load_state_dict(checkpoint)
        self.model = model
        print(f'Model loaded: {model}')

    def getEntropy(self, y):
        ensemble_probabilities = y.mean(0)
        entropy = Categorical(probs = ensemble_probabilities).entropy()
        return entropy


    def trainSeedModelClosedForm(self, rounds):
        # loss function
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)


        if rounds == 1:
            self.highest_unc = set(range(self.seed_sample))
            self.unexplored_data = self.unexplored.difference(self.highest_unc)



        uncertainty = []
        hook_handles = []
        save_output = Saveoutput(instances= self.instances, batch_size=self.batch_size, rounds = self.rounds)
        self.model.eval()
        for layer in self.model.kl_layers:
            hook_handles.append(layer.register_forward_hook(save_output))

        with torch.no_grad():
            for batch_index, (X, y) in enumerate(self.dataloader_train):
                batch_size = X.shape[0]
                save_output.batch_size = batch_size
                X = X.repeat(self.instances, 1, 1)
                y = y.repeat(self.instances, 1)
                X, y = X.to(device), y.to(device)
                y_pred = model(X)
                

                ensemble_outputs = y.reshape(self.instances, batch_size, 1) # reshape to instances x batch_size x 1 (output size)
                
                
                uncertainty.append(self.getEntropy(ensemble_outputs))
                
                for u in uncertainty:
                    print(f'Uncertainty: {u}')
                
                
        
            save_output.clear()
            save_output.counter = 0
            for handle in hook_handles:
                handle.remove()
            
            uncertainty = torch.cat(uncertainty)
            
            new_indices = torch.argsort(uncertainty, descending=True).tolist()

            # remove already selected data
            new_indices = [i for i in new_indices if i not in self.highest_unc]  

            # select the highest uncertainty data
            self.highest_unc.union(set(new_indices[:self.seed_sample]))
            self.unexplored_data = self.unexplored_data.difference(self.highest_unc)
            
            

    write_summary = LogSummary(name='Simple Model')




# use the class to run the active learning
active_learning = runActiveLearning(model, dataloader_train, dataloader_test, dataloader_val, epochs=100, rounds=1, learning_rate=0.001, batch_size=128, instances=1, highest_unc=10, seed_sample=1000)
    

Running Model:SimpleFFBNN(
  (fc1): KlLayers (4 -> 10)
  (fc2): KlLayers (10 -> 20)
  (fc3): KlLayers (20 -> 1)
)


In [122]:
active_learning.trainSeedModelClosedForm(rounds = 2)

Uncertainty: tensor([1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07,
        1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07,
        1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07,
        1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07,
        1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07,
        1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07,
        1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07,
        1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07,
        1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07,
        1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07,
        1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07,
        1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07, 1.1921e-07,
        1.1921e-07, 1.1921e

In [8]:
def trainSeedModelClosedForm(self):
    hook_handles = []
    save_output = saveOutput(self.instances, self.batch_size, 0)
    self.model.eval()
    for layer in self.model.layers:
        hook_handles.append(layer.register_forward_hook(save_output))
    
    with torch.no_grad():
        for batch_index, (X, y) in enumerate(self.data_train):
            batch_size = X.shape[0]
            save_output.batch_size = batch_size
            X = X.repeat(self.instances, 1)
            y = y.repeat(self.instances)
            X, y = X.to(self.device), y.to(self.device)
            ensemble_outputs = self.model(X)

        save_output.clear()
        save_output.counter = 0
        for handle in hook_handles:
            handle.remove()
